In [205]:
using Catlab
using Catlab.CategoricalAlgebra
using Catlab.Graphs
using Catlab.Graphs.BasicGraphs
using Catlab.Graphics
using Catlab.Graphics.Graphviz
using Catlab.Graphics.GraphvizGraphs
using Catlab.Graphs.PropertyGraphs
using Catlab.CategoricalAlgebra.CSets
using Catlab.CategoricalAlgebra.FinSets, Catlab.CategoricalAlgebra
using DataStructures
using Colors
using Random

In [206]:
draw(g) = to_graphviz(g, node_labels=true)

draw (generic function with 1 method)

In [207]:
struct Preorder
    carrier:: Set{String}
    relation :: SortedSet{Pair{String,String}}
    """Construct valid preorder by taking reflexive/transitive closure"""
    function Preorder(carrier:: Set{String}, rel :: Vector{Pair{String,String}})
      for (a,b) ∈ rel
        a ∈ carrier && b ∈ carrier || error("relation element not in carrier set")
      end
      relation = SortedSet(rel)
      for (a, b, c) in Iterators.product(carrier, carrier, carrier)
        if ((a => b) ∈ relation) && ((b => c) ∈ relation)
          push!(relation, a => c) # enforces relation is transitive
        end
      end
      for i in carrier
        push!(relation, i => i) # enforces that relation is reflexive
      end
      return new(carrier, relation)
    end
  end

In [208]:
function load_major(path::String)::Pair{Set{String}, Vector{Pair{String, String}}}
    file = open(path, "r")
    classes = Set{String}()
    relations = Vector{Pair{String, String}}()
    while !eof(file)
        line = readline(file)
        if length(strip(line, ' ')) != 0
            sp = split(line, ":")

            if length(sp) >= 1
                name = strip(sp[1], ' ')
                push!(classes, string(name))
            end

            if length(sp) >= 2
                reqs_str = strip(sp[2], ['[', ']', ' '])
                reqs = split(reqs_str, ",")
                
                for req in reqs
                    req = strip(req, ' ')
                    if length(req) > 0
                        if !in(req, classes)
                            push!(classes, req)
                        end
                        push!(relations, req => name)
                    end
                end
            end
        end
    end
    close(file)
    return classes => relations
end

load_major (generic function with 1 method)

In [209]:
cs = load_major("reqs/CS.reqs")

Set(["Humanities1", "Interdisciplinary-3", "Interdisciplinary-2", "Social/Behavioral1", "MAC2312", "ENC3246", "CNT4007", "CIS4930-1", "CIS4930-4", "COP3530"  …  "HumOrSocOrPhy1", "CIS4930-5", "MAS3114", "STA3032", "COP3502", "CDA3101", "CIS4930-2", "MAC2311", "CIS4930-3", "Social/Behavioral2"]) => ["MAC2311" => "MAC2312", "MAC2312" => "MAC2313", "MAC2312" => "MAS3114", "MAC2311" => "STA3032", "MAC2311" => "PHY2048/PHY2048L", "PHY2048/PHY2048L" => "PHY2049/PHY2049L", "MAC2313" => "PHY2049/PHY2049L", "MAC2311" => "COP3502", "MAC2311" => "COP3503", "COP3502" => "COP3503"  …  "COP4600" => "CNT4007", "COP3530" => "CIS4930-1", "COP3530" => "CIS4930-2", "COP3530" => "CIS4930-3", "COP3530" => "CIS4930-4", "COP3530" => "CIS4930-5", "Interdisciplinary-1" => "Interdisciplinary-2", "Interdisciplinary-2" => "Interdisciplinary-3", "Interdisciplinary-3" => "Interdisciplinary-4", "Interdisciplinary-4" => "Interdisciplinary-5"]

In [210]:
function top_sort(vertices::Set{String}, edges::Vector{Pair{String,String}}, seed::Int = 0)
    Random.seed!(seed)

    l = []
    q = Queue{String}()
    d = Dict{String, Int}()

    for v in vertices
        d[v] = 0
        for (a,b) in edges
            if v == b && a != b
                d[v]+=1
            end
        end
    end


    
    # randomizes the starting order of the queue
    for v in shuffle!(collect(vertices))
        if d[v] == 0
            enqueue!(q, v)
        end
    end


    while !isempty(q)
        v = dequeue!(q)
        push!(l, v)

        for (a,b) in shuffle!(collect(edges))
            if v == a && a != b
                d[b]-=1
                if d[b] == 0
                    enqueue!(q, b)
                end
            end
        end
    end
    
    return l
end

top_sort (generic function with 2 methods)

In [211]:
for class in top_sort(cs.first, cs.second)
    println(class)
end

MAC2311
Social/Behavioral1
EGS4034
ENC3246
Interdisciplinary-1
Humanities2
Social/Behavioral2
HumOrSocOrPhy1
Humanities1
COP3502
MAC2312
PHY2048/PHY2048L
STA3032
Interdisciplinary-2
COP3503
MAS3114
MAC2313
Interdisciplinary-3
COT3100
PHY2049/PHY2049L
Interdisciplinary-4
CDA3101
COP3530
Interdisciplinary-5
CIS4301
COP4600
CIS4930-1
CIS4930-3
COP4533
CIS4930-4
CIS4930-2
COP4020
CIS4930-5
CEN3031
CNT4007


In [212]:
cs.second

35-element Vector{Pair{String, String}}:
             "MAC2311" => "MAC2312"
             "MAC2312" => "MAC2313"
             "MAC2312" => "MAS3114"
             "MAC2311" => "STA3032"
             "MAC2311" => "PHY2048/PHY2048L"
    "PHY2048/PHY2048L" => "PHY2049/PHY2049L"
             "MAC2313" => "PHY2049/PHY2049L"
             "MAC2311" => "COP3502"
             "MAC2311" => "COP3503"
             "COP3502" => "COP3503"
                       ⋮
             "COP3530" => "CIS4930-1"
             "COP3530" => "CIS4930-2"
             "COP3530" => "CIS4930-3"
             "COP3530" => "CIS4930-4"
             "COP3530" => "CIS4930-5"
 "Interdisciplinary-1" => "Interdisciplinary-2"
 "Interdisciplinary-2" => "Interdisciplinary-3"
 "Interdisciplinary-3" => "Interdisciplinary-4"
 "Interdisciplinary-4" => "Interdisciplinary-5"

In [217]:
function find_one_offs(preorder::Preorder)
    one_offs = Vector()
    d = Dict()
    for class in preorder.carrier
        d[class] = true
    end
    for (a,b) in preorder.relation
        if a != b
            d[a] = false
            d[b] = false
        end
    end
    for (class, is_one_off) in d
        if is_one_off
            push!(one_offs, class)
        end
    end
    return Set(one_offs)
end

function generate_schedule(preorder::Preorder, num_semesters::Int=8, seed::Int=0)
    Random.seed!(seed)
    classes = preorder.carrier
    reqs = preorder.relation

    full_schedule = Vector()

    one_off_classes = find_one_offs(preorder)


    # build dictionary of in counts like in topological sort
    in_counts = Dict()
    for class in preorder.carrier
        if !(class in one_off_classes)
            in_counts[class] = 0
            for (a,b) in preorder.relation
                if class == b && a != b
                    in_counts[class]+=1
                end
            end
        end
    end


    # schedule classes with prereqs
    for index in 1:num_semesters
        semester_schedule = Vector()

        available_classes = Set(collect(in_counts))

        # find bottoms
        # marking them for deletion
        for (class, in_count) in shuffle!(collect(available_classes))
            if in_count == 0 && length(semester_schedule) < ceil(length(preorder.carrier)/num_semesters)
                push!(semester_schedule, class)
            end
        end

        # delete taken classes
        for class in semester_schedule
            for (a,b) in preorder.relation
                if class == a
                    in_counts[b]-=1
                end
            end
            delete!(in_counts, class)
        end
        push!(full_schedule, semester_schedule)
    end


    println(in_counts)
    # schedule classes with no prereqs
    # prioritizing even credit distribution
    while length(one_off_classes) > 0
        # find semester with min number of credits
        min_index = 1
        for (index, semester) in enumerate(full_schedule)
            if length(full_schedule[index]) < length(full_schedule[min_index])
                min_index = index
            end 
        end
        class_to_add = collect(one_off_classes)[1]
        push!(full_schedule[min_index], class_to_add)
        delete!(one_off_classes, class_to_add)
    end
    if length(one_off_classes) > 0 || length(collect(in_counts)) > 0
        throw("Cannot generate schedule!")
    end
    return full_schedule
end

generate_schedule (generic function with 3 methods)

In [218]:
cs_preorder = Preorder(cs.first, cs.second)
schedule = generate_schedule(cs_preorder, 8)

Dict{Any, Any}()


8-element Vector{Any}:
 Any["MAC2311", "Interdisciplinary-1", "Social/Behavioral1", "HumOrSocOrPhy1"]
 Any["PHY2048/PHY2048L", "MAC2312", "STA3032", "COP3502", "Interdisciplinary-2"]
 Any["MAS3114", "Interdisciplinary-3", "COP3503", "MAC2313"]
 Any["COT3100", "PHY2049/PHY2049L", "Interdisciplinary-4", "ENC3246"]
 Any["CDA3101", "Interdisciplinary-5", "COP3530", "EGS4034"]
 Any["CIS4930-3", "CIS4301", "CIS4930-5", "CIS4930-4", "COP4533"]
 Any["CEN3031", "COP4020", "CIS4930-2", "COP4600", "CIS4930-1"]
 Any["CNT4007", "Humanities1", "Humanities2", "Social/Behavioral2"]

In [219]:
schedule = generate_schedule(cs_preorder, 10)

Dict{Any, Any}()


10-element Vector{Any}:
 Any["MAC2311", "Interdisciplinary-1", "ENC3246"]
 Any["PHY2048/PHY2048L", "MAC2312", "STA3032", "COP3502"]
 Any["MAS3114", "Interdisciplinary-2", "COP3503", "MAC2313"]
 Any["COT3100", "Interdisciplinary-3", "PHY2049/PHY2049L"]
 Any["COP3530", "Interdisciplinary-4", "CDA3101"]
 Any["Interdisciplinary-5", "CIS4930-5", "CIS4930-2", "COP4600"]
 Any["CNT4007", "CIS4930-4", "COP4533", "CIS4930-1"]
 Any["CIS4930-3", "CIS4301", "COP4020", "CEN3031"]
 Any["Humanities1", "Humanities2", "EGS4034"]
 Any["Social/Behavioral1", "HumOrSocOrPhy1", "Social/Behavioral2"]

In [216]:
schedule = generate_schedule(cs_preorder, 6)

String: "Cannot generate schedule!"